<a href="https://colab.research.google.com/github/atejap05/pos_data_science_furg/blob/main/disciplinas/Machine_Learning_I/tarefas/dados_enem/Furg_ECD_09_Machine_Learning_I_Tarefa_Dados_do_ENEM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Curso de Especialização em Ciência de Dados - FURG
## Machine Learning I - Tarefa: Dados do ENEM
### Prof. Marcelo Malheiros

---

Esta tarefa é para você **observar** e **analisar** este processo de Machine Learning.

Adicionalmente, sugere-se que você também experimente com os dados e com os algoritmos, fazendo algumas das modificações indicadas em várias partes deste _notebook_.

Note que não é preciso escrever mais código, apenas modificar o código já fornecido.

Um questionário _online_ dentro da disciplina no AVA será disponibilizado para coletar sua análise. Este questionário será também uma das tarefas avaliativas desta disciplina.

# Problema

O problema aqui descrito é uma **tarefa de regressão**. Com base nos dados reais das escolas do Ensino Médio avaliadas pelo ENEM em 2014 e 2015, precisamos ajustar um **modelo de previsão** para ser capaz de prever as médias das provas de Matemática e de Redação, separadamente.

# Inicialização

Aqui importamos as bibliotecas fundamentais de Python para este _notebook_.

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import sklearn

In [ ]:
import tensorflow as tf
from tensorflow import keras

print('tensorflow:      versão', tf.__version__)
print('keras integrada: versão', keras.__version__)

# Conjunto de dados

Este _dataset_ deve ser baixado pelo analista diretamente do site do Instituto Nacional de Estudos e Pesquisas Educacionais Anísio Teixeira (INEP):

[Microdados do Exame Nacional do Ensino Médio por Escola](https://www.gov.br/inep/pt-br/acesso-a-informacao/dados-abertos/microdados/enem-por-escola)

Os dados estão presentes em um único arquivo no formato CSV na pasta `DADOS`, chamado `MICRODADOS_ENEM_ESCOLA.csv`.

A explicação sobre o layout dos dados e valores categóricos está dentro na pasta `DICIONÁRIO`, no arquivo `Dicionário_Microdados_Enem_Escola.xlsx`.

As descrições detalhadas do conjunto de dados estão na pasta `LEIA-ME e DOCUMENTOS TÉCNICOS`.

A leitura dos dados é feita a seguir, definido `;` como separador entre os campos. A opção `low_memory=False` é usada para fazer um exame mais detalhado do arquivo antes da leitura.

Note que para arquivos com palavras em Português é preciso tomar cuidado com a codifição dos campos de texto. Por padrão a codificação é `encoding='utf_8'`. Porém, se os acentos aparecem errados provavelmente a codificação precisa ser `encoding='latin_1'`, como foi o caso aqui.

**Não se esqueça de fazer o upload do arquivo `MICRODADOS_ENEM_ESCOLA.csv` para o Colaboratory, antes de rodar a célula a seguir.**

In [ ]:
# leitura dos conjuntos de dados

enem_original = pd.read_csv('MICRODADOS_ENEM_ESCOLA.csv', sep=';', encoding='latin_1', low_memory=False) # 

O conjunto inteiro dos dados foi colocado em um `DataFrame` (da biblioteca Pandas).

Vamos visualizar a seguir os primeiro cinco registros desta base:

In [ ]:
enem_original.head(5)

Não vamos detalhar cada campo, pois os detalhes estão disponível no arquivo de dicionário.

O importante aqui é notar que os dados abrangem diversos anos, estando **empilhados** nesta base. Então uma mesma escola, representada pelo código `CO_ESCOLA_EDUCACENSO` aparece em diversas linhas, uma para cada ano em que participou.

Então, por enquanto, usaremos o índice numérico automaticamente criado pela biblioteca Pandas (valores em negrito, à esquerda).

# Análise dos dados

Aqui vamos computar algumas estatísticas sobre a base original de dados. Estas estatísticas são importantes tanto para perceber quais operações de preprocessamento serão necessárias como para escolher quais _features_ vamos usar no treinamento.

Uma função muito útil para isso é a `.info()`, que mostra o tipo de dados e o número de valores presentes em cada coluna:

In [ ]:
enem_original.info()

Para este conjunto há um total de 172305 registros. Mas o número de escolas é muito menor, uma vez que esta base abrange dados de 2005 até 2015.

Além desta séries históricas estarem empilhadas, outro desafio desta base é que alguns atributos so valem para determinados anos, como `NU_MEDIA_OBJ` e `NU_MEDIA_TOT`. Isso implica em uma grande quantidade de valores faltantes, representados como `NaN` na visualização acima.

Vamos então iniciar removendo colunas que não são de interesse. Iremos descartar os atributos (colunas) sobre as unidade federativa e municípios das escolas, assim como todas as médias de provas que não sejam Matemática ou Redação.

In [ ]:
enem_original.drop(columns=['CO_UF_ESCOLA', 'SG_UF_ESCOLA', 'CO_MUNICIPIO_ESCOLA', 'NO_MUNICIPIO_ESCOLA', 
                   'NU_MEDIA_CN', 'NU_MEDIA_CH', 'NU_MEDIA_LP', 'NU_MEDIA_OBJ', 'NU_MEDIA_TOT'], inplace=True)
enem_original.shape

Note que fomos de 27 atributos para 18 agora.

Podemos também olhar estatísticas sobre os valores das colunas numéricas usando `.describe()':

In [ ]:
enem_original.describe()

Vale a pena também analisar as colunas que contém poucos valores, chamadas de categóricas, para identificar as classes presentes. Faremos isso chamando a função `.value_counts()` em cada uma.

In [ ]:
enem_original['NU_ANO'].value_counts().sort_index()

In [ ]:
# esta coluna foi lida como numérica, mas por ser um valor categórico vamos converter para string
enem_original['TP_DEPENDENCIA_ADM_ESCOLA'] = enem_original['TP_DEPENDENCIA_ADM_ESCOLA'].astype('str')
enem_original['TP_DEPENDENCIA_ADM_ESCOLA'].value_counts()

In [ ]:
# esta coluna foi lida como numérica, mas por ser um valor categórico vamos converter para string
enem_original['TP_LOCALIZACAO_ESCOLA'] = enem_original['TP_LOCALIZACAO_ESCOLA'].astype('str')
enem_original['TP_LOCALIZACAO_ESCOLA'].value_counts()

In [ ]:
enem_original['INSE'].value_counts()

In [ ]:
enem_original['PORTE_ESCOLA'].value_counts()

Queremos prever os atributos NU_MEDIA_MT (média da prova de Matemática) e NU_MEDIA_RED (média da prova de redação) para o último disponível, 2015. A própria escala das provas mudou, e de 2009 a 2015 vai de 0 a 1000. 

Como os atributos que desejamos prever são numéricos, temos uma tarefa de **regressão**. Isso afeta diretamente quais algoritmos podemos usar para fazer o treino, posteriormente. E também a arquitetura de rede neural a ser utilizada.

Então, como os valores a serem previstos são numericamente grandes, vamos utilizar como medida de desempenho o **erro absoluto médio** (_mean absolute error_, ou MAE). Isso é importante porque uma medida como RMSE é excessivamente sensível a variações maiores, e afetaria o próprio processo de treinamento.

# Reorganização dos dados

Aqui vale a pena filtrar os dados, retendo apenas os registros para os anos de 2014 e 2015.

Para isso vamos fazer primeiro uma operação de separação dos dados, com um `DataFrame` para 2014 e outro para 2015, apenas:

In [ ]:
enem_2014 = enem_original[enem_original['NU_ANO'] == 2014]
print(enem_2014.shape)

enem_2015 = enem_original[enem_original['NU_ANO'] == 2015]
print(enem_2015.shape)        

Precisamos então fazer uma operação de fusão (_merge_) entre as duas tabelas.

Isso é necessário para ter em uma única linha os atributos de uma mesma escola. Como teremos atributos de 2014 e 2015, podemos adicionar um prefixo para identificá-los na tabela resultante.

Outra característica importante desta fusão é que a amarração se dará pelo código único da escola, `CO_ESCOLA_EDUCACENSO`. Isso fará que apenas escolas que tenham participado das provas de 2014 e de 2015 estejam presentes na tabela resultante.

In [ ]:
enem = pd.merge(left=enem_2014, right=enem_2015, on='CO_ESCOLA_EDUCACENSO', suffixes=['_2014', '_2015'])
enem.set_index('CO_ESCOLA_EDUCACENSO', inplace=True)
enem.shape

In [ ]:
enem.head(5)

Aqui vamos retirar mais algumas colunas, deixando apenas as notas das provas de Matemática e Redação de 2014, e todos os dados restantes de 2015.

**Ajuste:** Pode ser interessante analisar futuramente se a manutenção de algum desses dados históricos também ajuda na melhoria do modelo. Para isso basta repetir a operação de _merge_ acima e então retirar do comando _drop_ abaixo o nome das colunas a serem mantidas. Estas novas colunas deverão ser selecionadas explicitamente na etapa posterior de pré-processamento dos dados.

In [ ]:
enem.drop(columns=['NU_ANO_2014', 'NU_ANO_2015', 'NO_ESCOLA_EDUCACENSO_2014',
                   'TP_DEPENDENCIA_ADM_ESCOLA_2014', 'TP_LOCALIZACAO_ESCOLA_2014', 'NU_MATRICULAS_2014', 
                   'NU_PARTICIPANTES_NEC_ESP_2014', 'NU_PARTICIPANTES_2014', 'NU_TAXA_PARTICIPACAO_2014',
                   'INSE_2014', 'PC_FORMACAO_DOCENTE_2014', 'NU_TAXA_PERMANENCIA_2014', 'NU_TAXA_APROVACAO_2014',
                   'NU_TAXA_REPROVACAO_2014', 'NU_TAXA_ABANDONO_2014', 'PORTE_ESCOLA_2014'], inplace=True)
enem.shape

Esta então é a nossa base final de trabalho, chamada simplesmente `enem`.

In [ ]:
enem.head(5)

In [ ]:
enem.info()

# Correlação entre atributos

Apenas para gerar uma intuição sobre os dados, vamos visualizar a correlação entre todos atributos e um dos alvos, `NU_MEDIA_MT_2015`.

Para isso usamos a função `.corr()` para calcular o coeficiente de correlação (também chamado de R de Pearson) entre cada par de atributos de um DataFrame. Note que aparecem apenas os **atributos numéricos**.

In [ ]:
# cálculo da matriz de correlação
corr = enem.corr()

# quanto cada atributo se correlaciona com o valor da prova de Matemática de 2015
corr['NU_MEDIA_MT_2015'].sort_values()

Note que é possível visualizar a tabela inteira, que cruza cada os atributos com todos os demais, como visualizado abaixo. A escala de cores é de um azul mais intenso para valores negativos (no mínimo -1) e de vermelhos mais intensos para valores positivos (no máximo 1). Valores próximos ao zero são também mais próximos do branco.

Ainda que haja correlação forte entre alguns atributos, para esta análise só interessam correlações envolvendo os atributos `NU_MEDIA_MT_2015` e `NU_MEDIA_RED_2015`.

In [ ]:
corr.style.background_gradient(axis=None, cmap='bwr').set_precision(2)

# Separação dos dados

Agora vamos separar cada conjunto de dados em duas partes: `features`, que são os atributos sobre os quais treinaremos o modelo, e `rotulos`, que contém os atributos a serem previstos.

In [ ]:
# rotulos
rotulos = enem[['NU_MEDIA_MT_2015', 'NU_MEDIA_RED_2015']]

# features
features = enem.drop(columns=['NU_MEDIA_MT_2015', 'NU_MEDIA_RED_2015'])

Vamos agora separar as _features_ e os rótulos em subconjuntos de treino e de testes.

Não vamos precisar de conjuntos de validação, pois iremos utilizar depois a estratégia de validação cruzada para garantir que o modelo não está fazendo _overfitting_.

In [ ]:
# separação em conjuntos de treino (80%) e de teste (20%)

from sklearn.model_selection import train_test_split
treino_features, teste_features, treino_rotulos, teste_rotulos = train_test_split(features, rotulos,
                                                                                  random_state=42, 
                                                                                  train_size=0.80)

É importante verificar o número de linhas e de colunas para cada subconjunto criado:

In [ ]:
print('treino_features:', treino_features.shape)
print('treino_rotulos: ', treino_rotulos.shape)
print('teste_features: ', teste_features.shape)
print('teste_rotulos:  ', teste_rotulos.shape)

In [ ]:
print(treino_features.iloc[0])

In [ ]:
print(treino_rotulos.iloc[0])

# Preprocessamento

Aqui faremos a etapa de preprocessamento, necessária para transformar os dados brutos em valores mais adequados para os algoritmos de Machine Learning.

In [ ]:
from sklearn.pipeline import Pipeline

In [ ]:
# aqui definimos uma etapa auxiliar do pipeline para atributos categóricos:
# esta substitui valores faltando pelo mais frequente de cada coluna
# (não se preocupe com os detalhes do código)

from sklearn.base import BaseEstimator, TransformerMixin

class MostFrequentImputer(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        self.most_frequent_ = pd.Series([X[c].value_counts().index[0] for c in X], index=X.columns)
        return self
    def transform(self, X, y=None):
        return X.fillna(self.most_frequent_)

Atributos numéricos (contínuos) e atributos categóricos (discretos) precisam ser processados separadamente.

O trecho a seguir define um _pipeline_ genérico, apenas para os atributos numéricos. A etapa usando `SimpleImputer(strategy='median')` preenche valores faltando com a mediana dos valores daquele atributo.

In [ ]:
# pipeline para atributos numéricos

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

num_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('std_scaler', StandardScaler()),
])

O trecho a seguir define um pipeline genérico, apenas para os atributos categóricos.

A etapa usando `MostFrequentImputer()` preenche valores faltando com o mais frequente dos valores daquele atributo. Já a etapa seguinte, usando `OneHotEncoder(sparse=False)` expande cada atributo que não é numérico para um conjunto de atributos binários. Esta precisa ser a última etapa desse _pipeline_.

In [ ]:
# pipeline para atributos categóricos

from sklearn.preprocessing import OneHotEncoder

cat_pipeline = Pipeline([
    ('imputer', MostFrequentImputer()),
    ('encoder', OneHotEncoder(sparse=False)),
])

A seguir usamos o recurso `ColumnTransformer` para aplicar determinados _pipelines_ para determinados atributos. Assim temos um _pipeline_ inteligente que processa os dados de uma só vez, mas executa operações diferentes conforme a natureza de cada atributo.

In [ ]:
# pipeline combinando atributos numéricos com atributos categóricos

from sklearn.compose import ColumnTransformer

full_pipeline = ColumnTransformer([
    ('num', num_pipeline, ['NU_MEDIA_MT_2014', 'NU_MEDIA_RED_2014',
                           'NU_MATRICULAS_2015', 
                           'NU_PARTICIPANTES_NEC_ESP_2015', 'NU_PARTICIPANTES_2015', 'NU_TAXA_PARTICIPACAO_2015',
                           'PC_FORMACAO_DOCENTE_2015', 'NU_TAXA_PERMANENCIA_2015', 'NU_TAXA_APROVACAO_2015',
                           'NU_TAXA_REPROVACAO_2015', 'NU_TAXA_ABANDONO_2015']),
    ('cat', cat_pipeline, ['TP_DEPENDENCIA_ADM_ESCOLA_2015', 'TP_LOCALIZACAO_ESCOLA_2015', 'INSE_2015',
                           'PORTE_ESCOLA_2015']),
])

Enfim, processamos os features de treino e de teste, gerando os conjuntos prontos para o treinamento. Note que para os dados de treino usamos `.fit_transform()`, enquanto para os de teste apenas `.transform()`.

In [ ]:
# preprocessamento
X_treino = full_pipeline.fit_transform(treino_features)
X_teste  = full_pipeline.transform(teste_features)
print(X_treino)

Aqui vamos extrair somente os dados brutos dos rótulos, e colocar em `y_treino`, que é o formato esperado pelos algoritmos de Machine Learning.

In [ ]:
y_treino = treino_rotulos.values
y_teste  = teste_rotulos.values
print(y_treino)

# Seleção do modelo

Aqui selecionamos um modelo para o aprendizado, baseado em um dos possíveis algoritmos de regressão. 

Como estamos criando um modelo com duas saídas (_multioutput_), para regressores simples precisamos usar o recurso `MultiOutputRegressor`. Este cria internamente um modelo separado para cada saída, mas os processos de treino, validação, teste e predição continuam os mesmos.

Já o modelo do tipo `RandomForestRegressor` é naturalmente capaz de gerar múltiplas saídas, então pode ser utilizado diretamente.

**Ajuste:** Depois de medir o desempenho do modelo você pode habilitar outros algoritmos. Note que apenas um  comando deve estar ativo (a linha de criação do `classificador`).

- Para o algoritmo `SVR` você pode mudar os hiperparâmetros `C` e `gamma='auto'`.

- Para o algoritmo `SGDRegressor` você pode mudar o hiperparâmetro `eta0`.

- Para o algoritmo `LinearRegression` não há hiperparâmetros a serem ajustados.

- Para o algoritmo `RandomForestRegressor` você pode aumentar o parâmetro `n_estimators`, por exemplo para 10 ou 100.

In [ ]:
from sklearn.multioutput import MultiOutputRegressor
from sklearn.svm import SVR
from sklearn.linear_model import SGDRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor

regressor = MultiOutputRegressor(SVR(C=1.0, gamma='scale'))

#regressor = MultiOutputRegressor(SGDRegressor(random_state=42, learning_rate='constant', eta0=0.0001))

#regressor = MultiOutputRegressor(LinearRegression())

#regressor = RandomForestRegressor(random_state=42, n_estimators=1)

# Treino e medida de desempenho

Vamos treinar e avaliar o regressor fazendo a **validação cruzada** do conjunto de treino. O parâmetro `cv=3` indica o número de dobras (ou _folds_), que é o número de vezes em que o conjunto é repartido, treinado e mensurado.

Como medida de desempenho vamos usar o erro absoluto médio. **Quanto menor esse erro, melhor.** Ele indica em média quantos pontos a mais ou menos as previsões das notas erram.

Como a validação cruzada devolve várias medidas, guardadas na lista `scores`, calculamos e exibimos uma média aritmética simples dessas medidas.

In [ ]:
# validação cruzada

from sklearn.model_selection import cross_val_score

%time scores = cross_val_score(regressor, X_treino, y_treino, cv=3, scoring='neg_mean_absolute_error')
print('scores: ' + ('{:.2f} ' * len(scores)).format(*(-scores)))
print('média:  {:.2f}'.format(-scores.mean()))

**Opcional:** Para melhorar o modelo existem diversas possibilidades de **ajuste** do modelo:

- A ação mais direta é simplesmente testar outros algoritmos, como mostrado na seção "Seleção do Modelo".

- Também é possível experimentar com os hiperparâmetros dos diversos algoritmos.

- Outra possibilidade é reduzir o número de _features_, simplesmente retirando atributos numéricos e categóricos do _pipeline_ `full_pipeline` na seção "Preprocessamento".

**Avançado**: Ainda é possível incluir mais _features_ do ano de 2014, o que deve ser feito na seção "Reorganização dos dados" e depois na seção "Preprocessamento".

# Teste final do modelo

Se você estiver satisfeito com a qualidade obtida na validação cruzada, então podemos fazer o teste final do modelo `regressor`.

Note que o treinamento do modelo é feito de fato aqui, com a chamada de `.fit()`. A validação cruzada feita anteriormente gera modelos temporários para as medidas e os descarta, mas não deixa o modelo treinado.

In [ ]:
# treinamento
regressor.fit(X_treino, y_treino)

# geração das previsões
y_teste_pred = regressor.predict(X_teste)
print(y_teste_pred)

Agora vamos avaliar o erro absoluto médio novamente, porém comparando o valores previstos com base nos dados de teste contra os rótulos reais para os mesmos dados de teste.

In [ ]:
# medida de desempenho
from sklearn.metrics import mean_absolute_error
score_teste = mean_absolute_error(y_teste, y_teste_pred)
print('score teste: {:.2f}'.format(score_teste))

Caso o modelo escolhido seja do tipo `RandomForestRegressor`, podemos inspecionar a importância aprendida para as _features_ do conjunto, o que dá um _insight_ importante sobre os dados. Para outros algoritmos não existem estas medidas.

In [ ]:
# lista a importância das features
if type(regressor) != RandomForestRegressor:
    print('AVISO: este tipo de regressor não permite visualizar a importância das features.')
    print('Se desejar, use como regressor o RandomForestRegressor na seção SELEÇÃO DO MODELO.')
else:
    # features listadas por ordem crescente de importância
    for score, name in sorted(zip(regressor.feature_importances_, treino_features.columns)):
        print('{:06.2%}'.format(score), name)

# Criando um modelo de rede neural

Aqui vamos criar uma rede neural de regressão, usando um modelo (ou arquitetura) do tipo sequencial. O modelo sequencial corresponde ao tipo mais simples de rede neural, onde uma sequência de camadas de neurônios é empilhada uma em cima da outra.

Para este tipo específico de tarefa de regressão, não selecionamos nenhuma função de ativação para a camada de saída. Ou seja, basta deixar os dados "brutos" sairem da rede neural.

Além disso, como queremos gerar a previsão para dois rótulos distintos, a camada de saída precisa ter **duas saídas**.

Observe os demais parâmetros para o processo de treinamento:

- O parâmetro `n_splits=2` define o número de quebras do conjunto de treinamento para a validação cruzada.

- A arquitetura da rede pode ser modificada, adicionando novas camadas ocultas ou mudando o númere de neurônio das mesmas.

- A função de perda é dada por `loss='mean_absolute_error'`, e precisa ser a mesma utilizada anteriormente em outros modelos, para que possamos comparar seu desempenho.

- A taxa de aprendizado é dada pelo hiperparâmetro `lr=0.01`, que também pode ser experimentada.

- É possivel usar a estratégia de _early stopping_ descomentando a linha `callbacks=[keras.callbacks.EarlyStopping(patience=5)],`.

- Pode-se ainda mudar o número de épocas com o hiperparâmetro `epochs=10` (experimente com valores maiores).

In [ ]:
# comandos para 'zerar' a biblioteca Keras e definir as sementes aleatórias
keras.backend.clear_session()
np.random.seed(42)
tf.random.set_seed(42)

# controle da validação cruzada
from sklearn.model_selection import KFold
kfold = KFold(random_state=42, shuffle=True, n_splits=2) # número de folds
scores = []
for fold_treino, fold_valid in kfold.split(X_treino, y_treino):
    # especificação do modelo
    model = keras.models.Sequential([
        keras.layers.Dense(30, activation='relu', input_shape=[X_treino.shape[1]]),
      # keras.layers.Dense(10, activation='relu'),
        keras.layers.Dense(2, activation=None)
    ])

    # especificação da função de perda e do algoritmo de otimização
    model.compile(loss='mean_absolute_error', optimizer=keras.optimizers.SGD(lr=0.01)) # taxa de aprendizado

    # treinamento
    print('------------------------------------------= FOLD =------------------------------------------')
    history = model.fit(X_treino[fold_treino], y_treino[fold_treino], epochs=10, # número de épocas
                        #callbacks=[keras.callbacks.EarlyStopping(patience=5)],
                        validation_data=(X_treino[fold_valid], y_treino[fold_valid]))
    scores.append(history.history['loss'][-1])
    
    # exibição das funções de perda de treino e de validação, para cada época (eixo horizontal)
    pd.DataFrame(history.history).iloc[2:].plot(figsize=(10, 4))
    plt.grid(True)
    plt.show()
    
print('scores: ' + ('{:.2f} ' * len(scores)).format(*scores))
print('média:  {:.2f}'.format(np.mean(scores)))

# Geração de previsões e teste final do modelo de rede neural

In [ ]:
# previsões computadas para três instâncias de teste
y_predi = model.predict(tf.constant(X_teste[:3]))
print('previsões: ', y_predi[0].round(2), y_predi[1].round(2), y_predi[2].round(2))
print('rótulos:   ', y_teste[0].round(2), y_teste[1].round(2), y_teste[2].round(2))

In [ ]:
# avaliação com conjunto de teste
score_teste = model.evaluate(X_teste, y_teste)
print('score teste: {:.2f}'.format(score_teste))